In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
from datetime import datetime
import torch
import json
import mbuild as mb
import gmso
from gmso.external.convert_networkx import to_networkx
import networkx as nx
from urllib.request import urlopen
from urllib.parse import quote
import torch.nn.functional as F

/Users/kieran/opt/miniconda3/envs/forward/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
COF_data_path = '/Users/kieran/iMoDELS-supplements/data/raw-data/everything.csv'
COF_data = pd.read_csv(COF_data_path, index_col=0)
COF_data = COF_data[['terminal_group_1','terminal_group_2','terminal_group_3', 'backbone', 'frac-1','frac-2','COF','intercept']]

molecules = glob.glob('/Users/kieran/terminal_groups_mixed/src/util/molecules/*.pdb')
molecules = list(set(molecules))
names2graph = {}
mol_smiles = {}
name2n_nodes = {}
name2xyz = {}
name2e_index = {}
name2mol = {}
name2x = {}
# name2edges

def replace_name_graph(chem_name):
    if chem_name == 'difluoromethyl' or chem_name == 'phenol' or chem_name == 'toluene':
        return
    else:
        return names2graph[chem_name]
def replace_n_nodes(chem_name):
    if chem_name == 'difluoromethyl' or chem_name == 'phenol' or chem_name == 'toluene':
        return
    else:
        return name2n_nodes[chem_name]
def replace_e_index(chem_name):
    if chem_name == 'difluoromethyl' or chem_name == 'phenol' or chem_name == 'toluene':
        return
    else:
        return name2e_index[chem_name]
def replace_node_att(chem_name):
    if chem_name == 'difluoromethyl' or chem_name == 'phenol' or chem_name == 'toluene':
        return
    else:
        x = torch.empty((len(elements),name2n_nodes[chem_name]), dtype=torch.int32)
        for i, p in enumerate(name2mol[chem_name].particles()):
            x[i] = element2vec[p]
        return x


def CIRconvert(ids):
    url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
    ans = urlopen(url).read().decode('utf8')
    return ans

for ids in set(COF_data['terminal_group_1']):
    try:
        mol_smiles[ids] = CIRconvert(ids)
    except:
        pass
mol_smiles['nitrophenyl'] = 'CC1=CC=C(C=C1)[N+]([O-])=O'
mol_smiles['isopropyl'] = 'CC(C)O'
mol_smiles['perfluoromethyl'] = 'CC(F)(F)F'
mol_smiles['fluorophenyl'] = 'CC1=CC=C(F)C=C1'
mol_smiles['carboxyl'] = '*C(=O)O'
mol_smiles['amino'] = 'CN'


names = ''.join(smiles.upper() for smiles in mol_smiles.values())
elements = [n for n in set(names) if n.isalpha()]


vecs = F.one_hot(torch.arange(0, 5), num_classes=5)
element2vec = {e:v for e, v in zip(elements, vecs)}
vec2element = {v:e for e, v in zip(elements, vecs)}

for m in molecules:
    mol_name = m.split('/')[-1].split('.')[0]
    if 'ch3' in mol_name:
        mol_name = mol_name.split('-')[0]
    mol = mb.load(m)
    G = to_networkx(mol.to_gmso())
    adj = nx.adjacency_matrix(G)
    names2graph[mol_name] = adj
    name2n_nodes[mol_name] = len(list(G.nodes))
    name2xyz[mol_name] = mol.xyz
    name2mol[mol_name] = mol
    e_index = torch.empty((2,mol.n_bonds), dtype=torch.int64)
    parts = {p:i for i, p in enumerate(mol.particles())}
    for i, b in enumerate(mol.bonds()):
        e_index[0,i] = parts[b[0]]
        e_index[1,i] = parts[b[1]]
    name2e_index[mol_name] = e_index
    x = torch.empty((mol.n_particles,len(elements)), dtype=torch.float32)
    for i, p in enumerate(mol.particles()):
        x[i] = element2vec[p.element.symbol]
    name2x[mol_name] = x
    
COF_data = COF_data.dropna()
COF_data = COF_data[(COF_data['frac-1']==.5)]
COF_data.reset_index()

/Users/kieran/opt/miniconda3/envs/forward/lib/python3.8/site-packages/gmso/core/subtopology.py:70: UserWarning: Setting a parent is potentially dangerous. Consider using Topology.add_subtopology instead
  warnings.warn(
/Users/kieran/opt/miniconda3/envs/forward/lib/python3.8/site-packages/gmso/core/topology.py:664: UserWarning: Non-parametrized site detected <Atom N, label: None id: 140375588421744>
  warnings.warn("Non-parametrized site detected {}".format(site))
/Users/kieran/opt/miniconda3/envs/forward/lib/python3.8/site-packages/gmso/core/topology.py:664: UserWarning: Non-parametrized site detected <Atom H, label: None id: 140375588422512>
  warnings.warn("Non-parametrized site detected {}".format(site))
/Users/kieran/opt/miniconda3/envs/forward/lib/python3.8/site-packages/gmso/core/topology.py:664: UserWarning: Non-parametrized site detected <Atom H, label: None id: 140375588424288>
  warnings.warn("Non-parametrized site detected {}".format(site))
/Users/kieran/opt/miniconda3/envs

,index,terminal_group_1,terminal_group_2,terminal_group_3,backbone,frac-1,frac-2,COF,intercept
0,0,acetyl,acetyl,acetyl,alkylsilane,0.5,0.5,0.140174,1.292272
1,1,acetyl,acetyl,carboxyl,alkylsilane,0.5,0.5,0.144711,3.382495
2,2,acetyl,acetyl,fluorophenyl,alkylsilane,0.5,0.5,0.120846,1.448256
3,3,acetyl,acetyl,hydroxyl,alkylsilane,0.5,0.5,0.149766,2.171407
4,4,acetyl,acetyl,isopropyl,alkylsilane,0.5,0.5,0.131951,0.748749
...,...,...,...,...,...,...,...,...,...
3344,3344,pyrrole,toluene,perfluoromethyl,alkylsilane,0.5,0.5,0.147257,0.866245
3345,3345,pyrrole,toluene,phenol,alkylsilane,0.5,0.5,0.132568,2.740356
3346,3346,pyrrole,toluene,phenyl,alkylsilane,0.5,0.5,0.135855,1.317521
3347,3347,pyrrole,toluene,pyrrole,alkylsilane,0.5,0.5,0.137062,1.261030


In [22]:
import torch
from torch_geometric.data import InMemoryDataset, download_url
from torch_geometric.data import Data


class TribologyDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return glob.glob('/Users/kieran/terminal_groups_mixed/src/util/molecules/*.pdb')

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        pass

    def process(self):
        # Read data into huge `Data` list.
        data_list = []
        
        for i, row in train_data.iterrows():  # Iterate in batches over the training dataset.
            if row.terminal_group_1 == 'difluoromethyl' or row.terminal_group_1 == 'phenol' or row.terminal_group_1 == 'toluene':
                continue
            x = name2x[row.terminal_group_1]
            edge_index = name2e_index[row.terminal_group_1]
            data = Data(x=x, edge_index=edge_index, y = torch.tensor((row.COF, row.intercept)))
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [23]:
from torch_geometric.loader import DataLoader

dataset = TribologyDataset(root='.')
dataset.shuffle()
train_dataset = dataset[:2000]
test_dataset = dataset[2000:]
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [24]:
from torch_geometric.nn import GraphConv
from torch_geometric.nn import global_mean_pool
from torch.nn import Linear
n_elements = 5
batch = 1

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(n_elements, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)

        x = self.lin(x)
        
        return x

In [25]:
from sklearn.metrics import mean_squared_error


model = GNN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x.float(), data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y.reshape((data.y.shape[0]//2,2)))  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    squared_error = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x.float(), data.edge_index, data.batch)
        res = data.y.reshape((data.y.shape[0]//2,2)) - out
        res = res.reshape(data.y.shape[0])
        
        squared_error.append(torch.pow(res, 2).sum())

    return sum(squared_error) / len(squared_error)


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 74.9562, Test Acc: 38.9213
Epoch: 002, Train Acc: 72.3086, Test Acc: 40.6556
Epoch: 003, Train Acc: 76.8346, Test Acc: 42.3394
Epoch: 004, Train Acc: 69.4987, Test Acc: 41.9122
Epoch: 005, Train Acc: 71.5917, Test Acc: 47.6802
Epoch: 006, Train Acc: 71.0240, Test Acc: 39.9935
Epoch: 007, Train Acc: 69.4971, Test Acc: 44.4481
Epoch: 008, Train Acc: 69.7451, Test Acc: 43.3808
Epoch: 009, Train Acc: 69.8085, Test Acc: 41.7425
Epoch: 010, Train Acc: 69.4497, Test Acc: 42.0304
Epoch: 011, Train Acc: 70.7725, Test Acc: 40.1579
Epoch: 012, Train Acc: 71.1061, Test Acc: 41.0543
Epoch: 013, Train Acc: 69.4659, Test Acc: 40.8428
Epoch: 014, Train Acc: 71.7521, Test Acc: 50.4391
Epoch: 015, Train Acc: 69.6558, Test Acc: 41.2431
Epoch: 016, Train Acc: 69.6674, Test Acc: 41.2915
Epoch: 017, Train Acc: 69.3623, Test Acc: 42.4014
Epoch: 018, Train Acc: 71.2649, Test Acc: 40.6982
Epoch: 019, Train Acc: 70.2603, Test Acc: 40.5351
Epoch: 020, Train Acc: 70.0245, Test Acc: 43.1226


In [9]:
for data in train_loader:
    print(data.x.dtype)
    print(data.edge_index.dtype)
    print(data.batch.dtype)
    print(data.y.size())
    break

torch.int64
torch.int64
torch.int64
torch.Size([128])


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GNN(hidden_channels=64)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 201):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train MSE: {train_acc:.4f}, Test MSE: {test_acc:.4f}')